In [1]:
import bitboards
import autoencoder
import pgn_reader as reader
import torch
import torch.nn as nn
import torch.optim as optim
import random
import math
import gc


gc.collect()
torch.cuda.empty_cache()

dsetSize, batch = 2**12, 2**8
#autoEncoder = autoencoder.autoencoder().cuda()
autoEncoder =  torch.load('autoencoder.pt').cuda()
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.Adam(autoEncoder.parameters())

class chessDataset(torch.utils.data.Dataset):
    def __init__(self, data_list):
        self.data = data_list
        
    def __getitem__(self, idx):
            return self.data[idx]
    
    def __len__(self):
        return len(self.data)


data = reader.get_dataset(dsetSize)
while data:
    random.shuffle(data)
    
    dataset = chessDataset(data[:math.floor(1/16*len(data))])
    testset = chessDataset(data[math.floor(1/16*len(data)):-1])

    DataLoader = torch.utils.data.DataLoader(dataset, batch_size = batch, shuffle = True)
    TestLoader = torch.utils.data.DataLoader(testset, batch_size = batch, shuffle = True)   
    
    autoEncoder.eval()
    test_loss = 0
    for x in TestLoader:
        x = x.cuda()
        y = autoEncoder(x).cuda()
        loss = criterion(y, x)
        test_loss += loss.item()
    print("Test loss: ", test_loss / len(testset))
        
    autoEncoder.train()
    training_loss = 0
    for x in DataLoader:
        optimizer.zero_grad()
        x = x.cuda()
        y = autoEncoder(x).cuda()
        loss = criterion(y, x)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
    print("Training loss: ", training_loss / len(dataset), end='\n\n')
    
    data = reader.get_dataset(dsetSize)
    
    torch.save(autoEncoder, 'autoencoderxallw3.pt')

Test loss:  3.640441628275757
Training loss:  3.8019691452556597

Test loss:  37.20832763798438
Training loss:  38.355742807054334

Test loss:  22.110897280210025
Training loss:  21.911518096923828

Test loss:  9.016855532467149
Training loss:  8.776115417480469

Test loss:  6.460134272389598
Training loss:  6.22179651260376

Test loss:  9.53078764928208
Training loss:  9.370807500419469

Test loss:  12.160723434130835
Training loss:  12.16809049999441

Test loss:  7.695222518302351
Training loss:  7.0783430071019415

Test loss:  8.188056229874473
Training loss:  7.594392879192646

Test loss:  8.177678034246819
Training loss:  7.969982458923577

Test loss:  5.693786514308923
Training loss:  5.472725459507534

Test loss:  5.371207216911995
Training loss:  5.975021031217755

Test loss:  5.53315459400355
Training loss:  5.757119455410324

Test loss:  4.952814827280596
Training loss:  4.974782670191092

Test loss:  4.776384855571546
Training loss:  5.020640210053767

Test loss:  4.43683928

KeyboardInterrupt: 